# Part 0

## Summary of Key Motivations and Contributions

* Most QA systems today answer literally, providing facts but not necessarily what's most helpful in conversation. For example, if you ask "Is there water on Mars?" a literal answer would be "Yes", but a friendly answer would be "Yes, but only in the form of ice caps near its poles.".

* Recent open-domain question answering (QA) datasets still fall short at two crucial desiderata.
    1. Existing QA datasets only test whether systems can find literal answers, not whether they understand what you really want to know
    2. Most of these datasets are crowd-sourced, which leaves them vulnerable to
    the problem of incentive misalignment between annotators and potential users.

* The contributions of the paper are:
    1. PRAGMATICQA dataset: They created the first conversational QA dataset that includes pragmatic answers (going beyond literal responses) and developed new metrics to measure how well AI systems can do pragmatic reasoning.
    2. Improved data collection: They designed a crowdsourcing method that fixes the incentive misalignment problem - where crowdworkers aren't motivated like real users would be. This produced more realistic, high-quality, and diverse conversation data.
    3. Current system analysis: They tested the dataset and showed it poses unique and important challenges that today's conversational QA systems can't handle well.

## Why this dataset challenging?

* PragmatiCQA is hard for NLP models because it tests whether models can have actual conversations, not just answer questions robotically. The dataset focuses on two key human conversational skills: anticipating what someone really wants to know (like explaining that Mars has water "but only as ice at the poles" instead of just saying "yes") and offering helpful context that keeps the conversation going (mentioning that 23 places in our solar system have water). What makes this super challenging is that AI has to guess what the person already knows, predict their likely follow-up questions, decide what extra info would actually be interesting - basically doing the kind of helpful, context responding that humans do but current NLP systems are terrible at, missing over 90% of the useful extra information humans naturally provide.

* It targets also few pragmatic phenomena
    - **Implicature**: Providing information that is implied or suggested, not just what is explicitly asked.
    - **Presupposition**: Recognizing and addressing background assumptions in the question.
    - **Relevance**: Selecting information that is most useful or interesting to the user, not just factually correct.
    - **Anticipation of follow-up**: Including information that preempts likely follow-up questions, making the conversation smoother and more natural.
    - **Disambiguation**: Clarifying ambiguous questions by inferring the user's intent.

## Dataset samples

1. **The Legend of Zelda**
    - **Question**: "What year did the Legend of Zelda come out?"
    - **Literal Answer**: "1986"
    - **Pragmatic Answer**: "The Legend of Zelda came out as early as 1986 for the Famicom in Japan, and was later released in the western world, including Europe and the US in 1987. Would you like to know about the story?"
    - **Enrichment**: The pragmatic answer not only provides the year but also adds information about the release in different places in the world and also anticipates a follow-up question and asking if the user wants to know more about the story.
2. **Po**
    - **Question**: "who is the main character ?"
    - **Literal Answer**: "Po"
    - **Pragmatic Answer**: "The main character of the movie is a panda named Po. He is also known as the Dragon Warrior. The Dragon Warrior is a prodigy of legend who is described in the first film to know the secret behind the Dragon Scroll. The Dragon Scroll is a legendary scroll that contains the secret to \"limitless power\" and had been written by Oogway."
    - **Enrichment**: The pragmatic answer provides the name of the main character, Po, but also adds context about his role as the Dragon Warrior and the significance of the Dragon Scroll, which is relevant to understanding the character's importance in the story.
3. **Snoop Dogg**
    - **Question**: "Who is Snoop Dogg?"
    - **Literal Answer**: "Snoop Dogg is an American rapper."
    - **Pragmatic Answer**: "Snoop Dogg is an American rapper who has sold over 23 millions albums nationally and over 35 million albums worldwide. Snoop Dogg sings and writes his own songs and produces records and even acts in films and television with his own personality."
    - **Enrichment**: The pragmatic answer provides the basic fact that Snoop Dogg is a rapper but also adds significant context about his achievements, including album sales and his multifaceted career in music and entertainment, which makes the answer more informative and engaging.
4. **Scuderia Ferrari**
    - **Question**: "Who is Scuderia Ferrari?"
    - **Literal Answer**: "Scuderia Ferrari is the racing division of Ferrari."
    - **Pragmatic Answer**: "Scuderia Ferrari is the racing division of Ferrari, and they are one of the most successsful teams in F1. They've won the drivers' title 15 times since the 1950s, and started making their own cars since 1947."
    - **Enrichment**: The pragmatic answer not only identifies Scuderia Ferrari as the racing division of Ferrari but also provides context about their success in Formula 1, including their championship wins and history, which adds depth to the answer.

5. **King Julien**
    - **Question**: "Who is King Julien?"
    - **Literal Answer**: "King Julien is a character in the Madagascar franchise."
    - **Pragmatic Answer**: "King Julien XIII is one of the main characters in the Madagascar franchise. He is the King of the Kingdom of Madagascar, ruling the lemur kingdom since his uncle, King Julien XII abdicated in King Me.",
    - **Enrichment**: The pragmatic answer provides the basic fact that King Julien is a character in the Madagascar franchise but also adds context about his role as the king of the lemur kingdom and his lineage, which enriches the understanding of his character within the story.